In [16]:
# TO DO:
# ensure places aren't repeated.  get them dupes out
# do iterative 50 at a time thingy

# markdown

# Identify if we can call config from separate spot in repo
# Extract subnational unit ie state where appropriate

# Print summary rollup of missed cities, and save log to txt file.
# https://www.pythontutorial.net/python-basics/python-create-text-file/

# WHy is the description part in a dict?

In [83]:
# Namespace
# Modules
import numpy as np
import pandas as pd
import requests
import string
# import os
from citipy import citipy
from pprint import pprint

# api keys, etc.  Is there a better way to import this than duplicating this in a second spot in the repo?
from config import weather_api_key

In [84]:
num_cities = 2000

# generate 2k random lat/long values
lats = np.random.uniform(low=-90.0, high=90.0, size=num_cities)
longs = np.random.uniform(low=-180.0, high=180.0, size=num_cities)
coords_zip = zip(lats,longs)
coords = list(coords_zip)

In [91]:
#instantiate base URL
base_url = "http://api.openweathermap.org/data/2.5/weather"

# instantiate list of results. will be comprised of dictionaries.
weather_data = list()

for i in range(50):
    # use citypy to get nearest cities
    city_obj = citipy.nearest_city(latitude=coords[i][0], longitude=coords[i][1])

    # Lower case strings for formal place names bother me.  So we're fixing that.
    city_name = string.capwords(city_obj.city_name)
    country_code = city_obj.country_code.upper()
    
    full_placename = f"{city_name},{country_code}"

    params={'appid':weather_api_key,
            'q':full_placename,
            'units':'imperial'
           }

    weather_response = requests.get(url=base_url,params=params)
    
    try:
        weather_json = weather_response.json()

        weather_data.append({
            "City" : city_name,
            "Country" : country_code,
            "Lat" : weather_json['coord']['lat'],
            "Long" : weather_json['coord']['lon'],
            "Max Temp" : weather_json['main']['temp_max'],
            "Pct Humidity" : weather_json['main']['humidity'],
            "Pct Cloudiness" : weather_json['clouds']['all'],
            "Wind Speed" : weather_json['wind']['speed'],
            "Description" : weather_json['weather'][0]['description']
        })
        
        print(f"Successfully processed {full_placename}.")
    except KeyError:
        print(f"API call failed to retrieve data for {full_placename}.  Error code: {weather_response.status_code}.")
        
print("Processing complete.  BUT I NEED A TIMESTAMP.")
print(f"Number of coordinates generated: {})
print(f"Successfully processed {len(weather_data)} total records.")

Successfully processed Norman Wells,CA.
Successfully processed Camapua,BR.
Successfully processed Varna,RU.
Successfully processed Umkomaas,ZA.
Successfully processed Vaini,TO.
Successfully processed Khatanga,RU.
Successfully processed Rosario Do Sul,BR.
Successfully processed New Norfolk,AU.
Successfully processed Bouake,CI.
Successfully processed Ilulissat,GL.
Successfully processed Norman Wells,CA.
Successfully processed Challakere,IN.
Successfully processed Nikolskoye,RU.
Successfully processed Avarua,CK.
Successfully processed Solnechnyy,RU.
Successfully processed Clyde River,CA.
Successfully processed Saskylakh,RU.
Successfully processed Santa Helena De Goias,BR.
Successfully processed Barrow,US.
API call failed to retrieve data for Barentsburg,SJ.  Error code: 404.
Successfully processed Terenos,BR.
Successfully processed Tarime,TZ.
Successfully processed Faanui,PF.
Successfully processed Carnarvon,AU.
Successfully processed Ushuaia,AR.
Successfully processed Arraial Do Cabo,BR.

In [92]:
weather_data_df = pd.DataFrame(weather_data)

weather_data_df.to_csv("WeatherPy_Database.csv")

weather_data_df

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description
0,Norman Wells,CA,65.2820,-126.8329,-11.16,70,90,5.75,light snow
1,Camapua,BR,-19.5314,-54.0439,69.84,95,100,6.89,overcast clouds
2,Varna,RU,53.3819,60.9747,-4.81,91,100,5.08,overcast clouds
3,Umkomaas,ZA,-30.2067,30.7978,72.59,94,100,4.79,light rain
4,Vaini,TO,-21.2000,-175.2000,86.16,84,75,8.05,broken clouds
5,Khatanga,RU,71.9667,102.5000,-24.11,96,10,4.05,clear sky
6,Rosario Do Sul,BR,-30.2583,-54.9142,75.92,42,1,16.26,clear sky
7,New Norfolk,AU,-42.7826,147.0587,79.75,20,88,4.59,overcast clouds
8,Bouake,CI,7.6895,-5.0218,73.92,48,16,4.27,few clouds
9,Ilulissat,GL,69.2167,-51.1000,-8.48,78,1,5.41,clear sky
